# DPL001-02 台股收盤資訊收集  
* 台股每日收盤資訊收集

In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
import requests
import duckdb

In [2]:
from finlab import data

In [ ]:
# 引用自建公用模組
sys.path.insert(0, str(Path.cwd().parent))
from proj_util_pkg.settings import ProjEnvSettings

from proj_util_pkg.finlab_api import finlab_manager as flm
from proj_util_pkg.common.duckdb_tool import insert_dataframe_to_duckdb

## 公用參數設定

In [4]:
# 欄數全展開
pd.set_option("display.max_columns", None)

In [ ]:
# finlab api 服務初始化
finlab = flm.FinlabManager()
data.force_cloud_download = False

## Function Block  


In [6]:
def combine_stock_dataframes_with_price_change(
        close, open, high, low, vol, pe_ratio, pb_ratio, dividend_yield, 
        recent_days_count=30, force_full_rebuild=False
    ):
    """
    將多個股票數據DataFrame合併為統一格式，並添加漲跌幅計算
    
    Parameters:
    所有數據參數都是 FinlabDataFrame，格式為: index=date, columns=symbol(股票代碼)
    recent_days_count (int): 要處理的最新N個交易日天數，預設30天（當force_full_rebuild=False時使用）
    force_full_rebuild (bool): 是否處理所有歷史資料，預設為False（僅處理最新資料以加快速度）
    
    Returns:
    DataFrame: 合併後的DataFrame，包含以下欄位:
    ['date', 'symbol', 'open', 'high', 'low', 'close', 'vol', 'pe_ratio', 'pb_ratio', 'dividend_yield', 'price_change']
    """
    
    def melt_df(df, value_name):
        """內部函數：將寬格式轉換為長格式"""
        df_reset = df.reset_index()
        df_melted = pd.melt(df_reset, id_vars=['date'], var_name='symbol', value_name=value_name)
        # 將 NaN 值補零
        df_melted[value_name] = df_melted[value_name].fillna(0)
        return df_melted
    
    def get_recent_data(df, days_count):
        """取得最新N個交易日的資料"""
        if df.empty:
            return df
        # 獲取最新的N個交易日
        recent_dates = df.index.sort_values(ascending=False)[:days_count]
        return df.loc[recent_dates]
    
    def calculate_price_change(df):
        """計算漲跌幅百分比"""
        # 先確保數據按symbol和date排序
        df_sorted = df.sort_values(['symbol', 'date']).copy()
        
        # 計算每個股票的前一日收盤價
        df_sorted['prev_close'] = df_sorted.groupby('symbol')['close'].shift(1)
        
        # 計算漲跌幅百分比 (今日收盤價 - 昨日收盤價) / 昨日收盤價 * 100
        df_sorted['price_change'] = (
            (df_sorted['close'] - df_sorted['prev_close']) / df_sorted['prev_close'] * 100
        ).round(2)
        
        # 移除輔助欄位
        df_sorted = df_sorted.drop('prev_close', axis=1)
        
        # 將第一個交易日的price_change設為0（沒有前一日資料）
        df_sorted['price_change'] = df_sorted['price_change'].fillna(0)
        
        return df_sorted
    
    # 準備所有要合併的DataFrame
    dataframes_to_process = {
        'close': close,
        'open': open,
        'high': high, 
        'low': low,
        'vol': vol,
        'pe_ratio': pe_ratio,
        'pb_ratio': pb_ratio,
        'dividend_yield': dividend_yield
    }
    
    # 根據設定決定處理範圍：僅處理最新資料 或 處理所有歷史資料
    if force_full_rebuild:
        print("處理所有歷史資料（force_full_rebuild=True）")
    else:
        print(f"僅處理最新 {recent_days_count} 個交易日的資料（預設模式，加快處理速度）")
        for name, df in dataframes_to_process.items():
            dataframes_to_process[name] = get_recent_data(df, recent_days_count)
    
    # 轉換所有DataFrame為長格式
    melted_dfs = {}
    for name, df in dataframes_to_process.items():
        melted_dfs[name] = melt_df(df, name)
        print(f"  {name}: {melted_dfs[name].shape[0]} 筆資料")
    
    # 從close開始合併
    result_df = melted_dfs['close'].copy()
    
    # 合併其他欄位
    merge_order = ['open', 'high', 'low', 'vol', 'pe_ratio', 'pb_ratio', 'dividend_yield']
    for column_name in merge_order:
        result_df = pd.merge(
            result_df, 
            melted_dfs[column_name], 
            on=['date', 'symbol'], 
            how='outer'
        )
    
    # 計算漲跌幅百分比
    print("正在計算漲跌幅百分比...")
    result_df = calculate_price_change(result_df)
    
    # 重新排列欄位順序
    desired_columns = ['date', 'symbol', 'open', 'high', 'low', 'close', 'vol', 'pe_ratio', 'pb_ratio', 'dividend_yield', 'price_change']
    result_df = result_df[desired_columns]
    
    print(f"合併完成，最終資料形狀: {result_df.shape}")
    
    return result_df


## 外部資料讀取  

In [7]:
# 讀取台股收盤價資訊
close = data.get("price:收盤價", save_to_storage=True)
open = data.get("price:開盤價", save_to_storage=True)
high = data.get("price:最高價", save_to_storage=True)
low = data.get("price:最低價", save_to_storage=True)
vol = data.get("price:成交股數", save_to_storage=True)
pe_ratio = data.get('price_earning_ratio:本益比', save_to_storage=True)
pb_ratio = data.get('price_earning_ratio:股價淨值比', save_to_storage=True)
dividend_yield = data.get('price_earning_ratio:殖利率(%)', save_to_storage=True)

### 🔧 使用方法

```python
# 1. 快速模式（預設，最新30天）
data = combine_stock_dataframes(close, open, high, low, vol, pe_ratio, pb_ratio, dividend_yield)

# 2. 自訂天數
data = combine_stock_dataframes(close, open, high, low, vol, pe_ratio, pb_ratio, dividend_yield, 
                               recent_days_count=60)

# 3. 強制重新合併所有資料
data = combine_stock_dataframes(close, open, high, low, vol, pe_ratio, pb_ratio, dividend_yield, 
                               force_full_rebuild=True)
```

In [ ]:
# 使用新的函數重新生成合併數據（預設使用最新30天資料）
print("=== 使用包含漲跌幅計算的函數合併數據（預設最新30天） ===")
final_stock_data = combine_stock_dataframes_with_price_change(
    close, open, high, low, vol, pe_ratio, pb_ratio, dividend_yield
    # , force_full_rebuild=True
)

# 將 date 欄位重新命名為 Date
final_stock_data = final_stock_data.rename(columns={'date': 'Date'})

print(f"函數生成的DataFrame shape: {final_stock_data.shape}")
print("前5筆資料:")
print(final_stock_data.tail())

## 資料留存ＤＢ

In [9]:
# 設定資料庫路徑
TWSTOCK_DATA_ROOT = os.environ.get("hist_data_path")
twstock_db_path = f"{TWSTOCK_DATA_ROOT}/twstock.duckdb"

In [10]:
# 連線資料庫
conn_duckdb = duckdb.connect(twstock_db_path)

In [11]:
table_name = "tw_stock_daily_txn"

In [ ]:
insert_row_count = insert_dataframe_to_duckdb(
    conn_duckdb, 
    final_stock_data, 
    table_name, 
    date_column='Date'
)

f"成功插入 {insert_row_count} 筆資料到 {table_name}"

In [ ]:
# 查詢表中所有資料
conn_duckdb.execute(f"SELECT * FROM {table_name} order by Date desc LIMIT 10").fetch_df()

In [ ]:
# 關閉資料庫連線
conn_duckdb.close()
print("資料庫連線已關閉")